In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import os

In [42]:
# Chemin du dossier contenant les images
image_root = './archive/Train'  # Chemin relatif vers les images

# Liste pour stocker les données
data = []

# Parcourir les sous-dossiers (chaque sous-dossier représente une classe)
for class_id, class_name in enumerate(os.listdir(image_root)):
    class_dir = os.path.join(image_root, class_name)
    
    # Vérifier que c'est un dossier
    if os.path.isdir(class_dir):
        # Parcourir les images dans le dossier de la classe
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_name, image_name)  # Chemin relatif de l'image
            data.append({'Path': image_path, 'ClassId': class_id})

# Créer un DataFrame à partir des données
df = pd.DataFrame(data)

# Sauvegarder le DataFrame en CSV
output_csv = './archive/image_labels.csv'
df.to_csv(output_csv, index=False)

print(f"Fichier CSV créé avec succès : {output_csv}")

Fichier CSV créé avec succès : ./archive/image_labels.csv


In [44]:
import os

img_path = "C:/Users/pc/Desktop/Twizzy/archive/Train/4/00004_00022_00021.png"  # Utilise le chemin complet
print(os.path.exists(img_path))  # Cela retournera True si le fichier existe, sinon False


True


In [45]:




# Charger le fichier CSV
train_csv = './archive/image_labels.csv'
train_labels = pd.read_csv(train_csv)

# Transformation des images (si nécessaire)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Créer un dataset personnalisé
class ImageDataset(Dataset):
    def __init__(self, root, labels, transform=None):
        self.root = root
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.labels.iloc[idx]['Path'])
        image = Image.open(img_path).convert('RGB')  # Ouvrir l'image et la convertir en RGB
        label = self.labels.iloc[idx]['ClassId']  # Récupérer l'étiquette

        if self.transform:
            image = self.transform(image)  # Appliquer les transformations (si présentes)

        return image, label

# Créer le dataset
train_dataset = ImageDataset(image_root, train_labels, transform=transform)

# Créer le DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)

# Vérifier les premières lignes du DataFrame des labels
print(train_labels.head())

# Vérifier si le DataLoader fonctionne
for images, labels in train_loader:
    print('Dimensions des images dans le batch:', images.shape)
    print('Dimensions des labels dans le batch:', labels.shape)
    break


                      Path  ClassId
0  1\00001_00000_00000.png        0
1  1\00001_00000_00001.png        0
2  1\00001_00000_00002.png        0
3  1\00001_00000_00003.png        0
4  1\00001_00000_00004.png        0
Dimensions des images dans le batch: torch.Size([128, 3, 64, 64])
Dimensions des labels dans le batch: torch.Size([128])


In [46]:
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        
        # Première couche convolutive (64x64 -> 32x32)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Pooling (32x32 -> 16x16)
        
        # Deuxième couche convolutive (16x16 -> 8x8)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Couche entièrement connectée
        self.fc1 = nn.Linear(64 * 16 * 16, 512)  # 64 canaux * 16x16 dimensions
        self.fc2 = nn.Linear(512, num_classes)  # Sortie avec 'num_classes' classes
    
    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))  # Appliquer ReLU après convolution
        x = self.pool(nn.ReLU()(self.conv2(x)))  # Appliquer ReLU après convolution
        
        x = x.view(-1, 64 * 16 * 16)  # Aplatir les données pour la couche entièrement connectée
        x = nn.ReLU()(self.fc1(x))  # Appliquer ReLU après la première couche entièrement connectée
        x = self.fc2(x)  # Sortie avec 'num_classes' classes
        
        return x

# Initialisation du modèle avec le nombre de classes
num_classes = len(train_labels['ClassId'].unique())
model = SimpleCNN(num_classes)

# Fonction de perte et optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement du modèle
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Passer en mode entraînement
    running_loss = 0.0
    
    for images, labels in train_loader:
        optimizer.zero_grad()  # Réinitialiser les gradients
        
        # Passer les images à travers le modèle
        outputs = model(images)
        
        # Calcul de la perte
        loss = criterion(outputs, labels)
        
        # Calculer les gradients et mettre à jour les poids
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Époque {epoch + 1}/{num_epochs}, Perte : {running_loss / len(train_loader)}")

print("Entraînement terminé")


Époque 1/10, Perte : 1.4078203161557516
Époque 2/10, Perte : 0.8343416889508565
Époque 3/10, Perte : 0.5062174956003825
Époque 4/10, Perte : 0.2944492772221565
Époque 5/10, Perte : 0.15418922975659372
Époque 6/10, Perte : 0.08815173817177614
Époque 7/10, Perte : 0.07166428851584593
Époque 8/10, Perte : 0.05063832765445113
Époque 9/10, Perte : 0.03751457060376803
Époque 10/10, Perte : 0.030459489362935226
Entraînement terminé
